In [1]:
import os
from pathlib import Path

import pandas as pd

import ad_server
import distributions
import simulation

In [2]:
windows = 24 * 60
requests = 2_000

base_distribution = distributions.custom_dist(windows)
requests_distribution = distributions.traffic_dist(base_distribution, requests)
campaigns = [ad_server.Campaign(1, 10_000, 10), ad_server.Campaign(2, 11_000, 11)]
pacing = ad_server.ThrottledPacing(base_distribution, 2 * 60)
# select_win = pacing.first_price_auction
select_win = ad_server.second_price_auction
process = ad_server.AdServer(pacing, select_win, campaigns)

In [3]:
sim = simulation.Simulation(requests_distribution, process)

In [4]:
df = pd.DataFrame(e.to_dict() for e in sim.run())
df.index.name = "ord"

In [5]:
path = Path("data/results/throttled.csv")
os.makedirs(path.parent, exist_ok=True)
df.to_csv(path)

In [6]:
path = Path("data/request_distribution.csv")
os.makedirs(path.parent, exist_ok=True)
s = pd.Series(requests_distribution)
s.index.name = "window"
s.name = "requests"
s.to_csv(path)